In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

import tensorflow as tf

from scipy.stats import truncnorm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import tqdm
import time
import os.path
import pdb
import argparse
import sys
from shapes_loader import *
from base_model import *

DEVICE = 'cuda'

In [2]:
# sample homographies


In [2]:
# warp image give


In [3]:
# parameter 
threshold = 0.05

# load in model
magic_leap_model = SuperPointNet().to(DEVICE)
model = torch.load('./Fri Nov  9 15:54:21 2018/e_405_a_10.9042.model').to(DEVICE)
magic_leap_model.load_state_dict(torch.load('superpoint_v1.pth'))


# load in images
img_dir = '../test_coco/'
imgsList = os.listdir(img_dir)
clr_imgs = [Image.open(img_dir+img).resize((400,300),Image.ANTIALIAS) for img in imgsList]
clr_imgs = clr_imgs[0]
imgs = torch.tensor(np.asarray([np.array(Image.open(img_dir+img).resize((400,300),Image.ANTIALIAS).convert('1')) for img in imgsList], dtype='float')).to(DEVICE)  
imgs = imgs[0:2]

print('original image shape: ', imgs.shape)
h,w = imgs[0].shape

#ipt_1, desc_1 = model(imgs.float().unsqueeze(1).to(DEVICE))
ipt_2, desc_2 = magic_leap_model(imgs.float().unsqueeze(1).to(DEVICE))

#ipt bnum x 65 x hc x wc
bnum, dims, hc, wc = ipt_2.shape
print('ML model output shape: ', ipt_2.shape)
print('ML model output shape: ', desc_2.shape)

original image shape:  torch.Size([2, 300, 400])
ML model output shape:  torch.Size([2, 65, 37, 50])
ML model output shape:  torch.Size([2, 256, 37, 50])


In [29]:
# homography adaptation
counts = torch.ones(ipt_2.shape)

print(ipt_2.shape)
probs = ipt_2.unsqueeze(-1)
counts = counts.unsqueeze(-1)
images = imgs.unsqueeze(-1)
shape = images.shape[:-1]
print(shape)


torch.Size([2, 65, 37, 50])
torch.Size([2, 300, 400])


In [34]:
# try tensorflow homography
shape = tf.shape(images)[1:3]
homography = sample_homography_tf(shape)

In [91]:
perspective=True
scaling=True
rotation=True
translation=True
n_scales=5
n_angles=25
scaling_amplitude=0.25
perspective_amplitude_x=0.2
perspective_amplitude_y=0.2 
patch_ratio=0.85
max_angle=1.57
allow_artifacts=False
translation_overflow=0.

# Corners of the output image
pts1 = torch.Tensor([[0., 0.], [0., 1.], [1., 1.], [1., 0.]])
# Corners of the input patch
margin = (1 - patch_ratio) / 2
pts2 = margin + torch.Tensor([[0, 0], [0, patch_ratio],
                             [patch_ratio, patch_ratio], [patch_ratio, 0]])

# Random perspective and affine perturbations
if True:
    if not False:
        perspective_amplitude_x = min(perspective_amplitude_x, margin)
        perspective_amplitude_y = min(perspective_amplitude_y, margin)
    
    # create truncated normal distribution
    perspective_displacement = truncated_normal(-perspective_amplitude_y, perspective_amplitude_y, perspective_amplitude_y/2)[0]
    h_displacement_left = truncated_normal(-perspective_amplitude_x, perspective_amplitude_x, perspective_amplitude_x/2)[0]
    h_displacement_right = truncated_normal(-perspective_amplitude_x, perspective_amplitude_x, perspective_amplitude_x/2)[0]
    pts2 += torch.Tensor([[h_displacement_left, perspective_displacement], [h_displacement_left, -perspective_displacement],
                          [h_displacement_right, perspective_displacement], [h_displacement_right, -perspective_displacement]])

print('pts2 after perspective...:', pts2)
# Random scaling
# sample several scales, check collision with borders, randomly pick a valid one
if scaling:
    scales = torch.ones([1+n_scales])
    scales[1:] = torch.from_numpy(truncated_normal(-scaling_amplitude, scaling_amplitude, scaling_amplitude/2, my_mean=1, sz=n_scales))
    center = torch.mean(pts2, dim=0, keepdim=True)
    scales = scales.unsqueeze(1).unsqueeze(1)
    scaled = (pts2-center).unsqueeze(0) * scales + center
#     temp = (pts2-center).unsqueeze(0)
#     print('center',center.shape)
#     print('temp',temp.shape)
#     print('scales', scales.shape)
#     print('scaled', scaled)
    
    if allow_artifacts:
        valid = torch.arange(1,n_scales+1) # all scales are valid except scale=1
    else:
        scaled_boolean = ( (scaled >= 0.) & (scaled < 1.) )
        valid = ( (logits).sum(dim=(1,2)) == 8 ).nonzero().squeeze(1) # get the index of valid
    # get the index
    idx = valid[torch.randint(low=0, high=6, size=(1,),dtype=torch.int32)[0]]
    pts2 = scaled[idx]
#     print('scaled', scaled)
#     print('idx', idx)

print("pts2 after scaling: ", pts2)

pts2 after perspective...: tensor([[0.0694, 0.1026],
        [0.0694, 0.8974],
        [0.8904, 0.9526],
        [0.8904, 0.0474]])
pts2 after scaling:  tensor([[0.3887, 0.4117],
        [0.3887, 0.5883],
        [0.5712, 0.6006],
        [0.5712, 0.3994]])


In [73]:
torch.randint(low=0, high=6, size=(1,),dtype=torch.int32)[0]

tensor(0, dtype=torch.int32)

In [22]:
def truncated_normal(myclip_a, myclip_b, my_std, my_mean=0., sz=1):
    """
    Generate random numbers from a truncated normal distribution
    """
    a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
    return truncnorm.rvs(a, b, my_mean, my_std, size=sz)

In [26]:
def sample_homography(
        shape, perspective=True, scaling=True, rotation=True, translation=True,
        n_scales=5, n_angles=25, scaling_amplitude=0.25, perspective_amplitude_x=0.2,
        perspective_amplitude_y=0.2, patch_ratio=0.85, max_angle=1.57,
        allow_artifacts=False, translation_overflow=0.):
    """Sample a random valid homography.

    Computes the homography transformation between a random patch in the original image
    and a warped projection with the same image size.
    As in `tf.contrib.image.transform`, it maps the output point (warped patch) to a
    transformed input point (original patch).
    The original patch, which is initialized with a simple half-size centered crop, is
    iteratively projected, scaled, rotated and translated.

    Arguments:
        shape: A rank-2 `Tensor` specifying the height and width of the original image.
        perspective: A boolean that enables the perspective and affine transformations.
        scaling: A boolean that enables the random scaling of the patch.
        rotation: A boolean that enables the random rotation of the patch.
        translation: A boolean that enables the random translation of the patch.
        n_scales: The number of tentative scales that are sampled when scaling.
        n_angles: The number of tentatives angles that are sampled when rotating.
        scaling_amplitude: Controls the amount of scale.
        perspective_amplitude_x: Controls the perspective effect in x direction.
        perspective_amplitude_y: Controls the perspective effect in y direction.
        patch_ratio: Controls the size of the patches used to create the homography.
        max_angle: Maximum angle used in rotations.
        allow_artifacts: A boolean that enables artifacts when applying the homography.
        translation_overflow: Amount of border artifacts caused by translation.

    Returns:
        A `Tensor` of shape `[1, 8]` corresponding to the flattened homography transform.
    """

    # Corners of the output image
    pts1 = torch.Tensor([[0., 0.], [0., 1.], [1., 1.], [1., 0.]])
    # Corners of the input patch
    margin = (1 - patch_ratio) / 2
    pts2 = margin + torch.Tensor([[0, 0], [0, patch_ratio],
                                 [patch_ratio, patch_ratio], [patch_ratio, 0]])
    
    # Random perspective and affine perturbations
    if perspective:
        if not allow_artifacts:
            perspective_amplitude_x = min(perspective_amplitude_x, margin)
            perspective_amplitude_y = min(perspective_amplitude_y, margin)

        # create truncated normal distribution
        perspective_displacement = truncated_normal(-perspective_amplitude_y, perspective_amplitude_y, perspective_amplitude_y/2)
        h_displacement_left = truncated_normal(-perspective_amplitude_x, perspective_amplitude_x, perspective_amplitude_x/2)
        h_displacement_right = truncated_normal(-perspective_amplitude_x, perspective_amplitude_x, perspective_amplitude_x/2)
        pts2 += torch.Tensor([[h_displacement_left, perspective_displacement], [h_displacement_left, -perspective_displacement],
                              [h_displacement_right, perspective_displacement], [h_displacement_right, -perspective_displacement]])

    # Random scaling
    # sample several scales, check collision with borders, randomly pick a valid one
    if scaling:
        scales = torch.ones([1+n_scales])
        scales[1:] = torch.from_numpy(truncated_normal(-scaling_amplitude, scaling_amplitude, scaling_amplitude/2, my_mean=1, sz=n_scales))
        center = torch.mean(pts2, dim=0, keepdim=True)
        scales = scales.unsqueeze(1).unsqueeze(1)
        scaled = (pts2-center).unsqueeze(0) * scales + center
    #     temp = (pts2-center).unsqueeze(0)
    #     print('center',center.shape)
    #     print('temp',temp.shape)
    #     print('scales', scales.shape)
    #     print('scaled', scaled)

        if allow_artifacts:
            valid = torch.arange(1,n_scales+1) # all scales are valid except scale=1
        else:
            scaled_boolean = ( (scaled >= 0.) & (scaled < 1.) )
            valid = ( (logits).sum(dim=(1,2)) == 8 ).nonzero().squeeze(1) # get the index of valid
        # get the index
        idx = valid[torch.randint(low=0, high=6, size=(1,),dtype=torch.int32)[0]]
        pts2 = scaled[idx]
    #     print('scaled', scaled)
    #     print('idx', idx)

    # Random translation
    if translation:
        t_min, t_max = tf.reduce_min(pts2, axis=0), tf.reduce_min(1 - pts2, axis=0)
        if allow_artifacts:
            t_min += translation_overflow
            t_max += translation_overflow
        pts2 += tf.expand_dims(tf.stack([tf.random_uniform((), -t_min[0], t_max[0]),
                                         tf.random_uniform((), -t_min[1], t_max[1])]),
                               axis=0)

    # Random rotation
    # sample several rotations, check collision with borders, randomly pick a valid one
    if rotation:
        angles = tf.lin_space(tf.constant(-max_angle), tf.constant(max_angle), n_angles)
        angles = tf.concat([angles, [0.]], axis=0)  # in case no rotation is valid
        center = tf.reduce_mean(pts2, axis=0, keepdims=True)
        rot_mat = tf.reshape(tf.stack([tf.cos(angles), -tf.sin(angles), tf.sin(angles),
                                       tf.cos(angles)], axis=1), [-1, 2, 2])
        rotated = tf.matmul(
                tf.tile(tf.expand_dims(pts2 - center, axis=0), [n_angles+1, 1, 1]),
                rot_mat) + center
        if allow_artifacts:
            valid = tf.range(n_angles)  # all angles are valid, except angle=0
        else:
            valid = tf.where(tf.reduce_all((rotated >= 0.) & (rotated < 1.),
                                           axis=[1, 2]))[:, 0]
        idx = valid[tf.random_uniform((), maxval=tf.shape(valid)[0], dtype=tf.int32)]
        pts2 = rotated[idx]

    # Rescale to actual size
    shape = tf.to_float(shape[::-1])  # different convention [y, x]
    pts1 *= tf.expand_dims(shape, axis=0)
    pts2 *= tf.expand_dims(shape, axis=0)

    def ax(p, q): return [p[0], p[1], 1, 0, 0, 0, -p[0] * q[0], -p[1] * q[0]]

    def ay(p, q): return [0, 0, 0, p[0], p[1], 1, -p[0] * q[1], -p[1] * q[1]]

    a_mat = tf.stack([f(pts1[i], pts2[i]) for i in range(4) for f in (ax, ay)], axis=0)
    p_mat = tf.transpose(tf.stack(
        [[pts2[i][j] for i in range(4) for j in range(2)]], axis=0))
    homography = tf.transpose(tf.matrix_solve_ls(a_mat, p_mat, fast=True))
    return homography

In [33]:
def sample_homography_tf(
        shape, perspective=True, scaling=True, rotation=True, translation=True,
        n_scales=5, n_angles=25, scaling_amplitude=0.25, perspective_amplitude_x=0.2,
        perspective_amplitude_y=0.2, patch_ratio=0.85, max_angle=1.57,
        allow_artifacts=False, translation_overflow=0.):
    """Sample a random valid homography.

    Computes the homography transformation between a random patch in the original image
    and a warped projection with the same image size.
    As in `tf.contrib.image.transform`, it maps the output point (warped patch) to a
    transformed input point (original patch).
    The original patch, which is initialized with a simple half-size centered crop, is
    iteratively projected, scaled, rotated and translated.

    Arguments:
        shape: A rank-2 `Tensor` specifying the height and width of the original image.
        perspective: A boolean that enables the perspective and affine transformations.
        scaling: A boolean that enables the random scaling of the patch.
        rotation: A boolean that enables the random rotation of the patch.
        translation: A boolean that enables the random translation of the patch.
        n_scales: The number of tentative scales that are sampled when scaling.
        n_angles: The number of tentatives angles that are sampled when rotating.
        scaling_amplitude: Controls the amount of scale.
        perspective_amplitude_x: Controls the perspective effect in x direction.
        perspective_amplitude_y: Controls the perspective effect in y direction.
        patch_ratio: Controls the size of the patches used to create the homography.
        max_angle: Maximum angle used in rotations.
        allow_artifacts: A boolean that enables artifacts when applying the homography.
        translation_overflow: Amount of border artifacts caused by translation.

    Returns:
        A `Tensor` of shape `[1, 8]` corresponding to the flattened homography transform.
    """

    pts1 = tf.stack([[0., 0.], [0., 1.], [1., 1.], [1., 0.]], axis=0)
    # Corners of the input patch
    margin = (1 - patch_ratio) / 2
    pts2 = margin + tf.constant([[0, 0], [0, patch_ratio], [patch_ratio, patch_ratio], [patch_ratio, 0]], tf.float32)

    
    # Random perspective and affine perturbations
    if perspective:
        if not allow_artifacts:
            perspective_amplitude_x = min(perspective_amplitude_x, margin)
            perspective_amplitude_y = min(perspective_amplitude_y, margin)

        perspective_displacement = tf.truncated_normal([1], 0., perspective_amplitude_y/2)
        h_displacement_left = tf.truncated_normal([1], 0., perspective_amplitude_x/2)
        h_displacement_right = tf.truncated_normal([1], 0., perspective_amplitude_x/2)
        pts2 += tf.stack([tf.concat([h_displacement_left, perspective_displacement], 0),
                          tf.concat([h_displacement_left, -perspective_displacement], 0),
                          tf.concat([h_displacement_right, perspective_displacement], 0),
                          tf.concat([h_displacement_right, -perspective_displacement], 0)])

    # Random scaling
    # sample several scales, check collision with borders, randomly pick a valid one
    if scaling:
        scales = tf.concat(
                [[1.], tf.truncated_normal([n_scales], 1, scaling_amplitude/2)], 0)
        center = tf.reduce_mean(pts2, axis=0, keepdims=True)
        scaled = tf.expand_dims(pts2 - center, axis=0) * tf.expand_dims(
                tf.expand_dims(scales, 1), 1) + center
        if allow_artifacts:
            valid = tf.range(n_scales)  # all scales are valid except scale=1
        else:
            valid = tf.where(tf.reduce_all((scaled >= 0.) & (scaled < 1.), [1, 2]))[:, 0]
            
        idx = valid[tf.random_uniform((), maxval=tf.shape(valid)[0], dtype=tf.int32)]
        pts2 = scaled[idx]

    # Random translation
    if translation:
        t_min, t_max = tf.reduce_min(pts2, axis=0), tf.reduce_min(1 - pts2, axis=0)
        if allow_artifacts:
            t_min += translation_overflow
            t_max += translation_overflow
        pts2 += tf.expand_dims(tf.stack([tf.random_uniform((), -t_min[0], t_max[0]),
                                         tf.random_uniform((), -t_min[1], t_max[1])]),
                               axis=0)

    # Random rotation
    # sample several rotations, check collision with borders, randomly pick a valid one
    if rotation:
        angles = tf.lin_space(tf.constant(-max_angle), tf.constant(max_angle), n_angles)
        angles = tf.concat([angles, [0.]], axis=0)  # in case no rotation is valid
        center = tf.reduce_mean(pts2, axis=0, keepdims=True)
        rot_mat = tf.reshape(tf.stack([tf.cos(angles), -tf.sin(angles), tf.sin(angles),
                                       tf.cos(angles)], axis=1), [-1, 2, 2])
        rotated = tf.matmul(
                tf.tile(tf.expand_dims(pts2 - center, axis=0), [n_angles+1, 1, 1]),
                rot_mat) + center
        if allow_artifacts:
            valid = tf.range(n_angles)  # all angles are valid, except angle=0
        else:
            valid = tf.where(tf.reduce_all((rotated >= 0.) & (rotated < 1.),
                                           axis=[1, 2]))[:, 0]
        idx = valid[tf.random_uniform((), maxval=tf.shape(valid)[0], dtype=tf.int32)]
        pts2 = rotated[idx]

    # Rescale to actual size
    shape = tf.to_float(shape[::-1])  # different convention [y, x]
    pts1 *= tf.expand_dims(shape, axis=0)
    pts2 *= tf.expand_dims(shape, axis=0)

    def ax(p, q): return [p[0], p[1], 1, 0, 0, 0, -p[0] * q[0], -p[1] * q[0]]

    def ay(p, q): return [0, 0, 0, p[0], p[1], 1, -p[0] * q[1], -p[1] * q[1]]

    a_mat = tf.stack([f(pts1[i], pts2[i]) for i in range(4) for f in (ax, ay)], axis=0)
    p_mat = tf.transpose(tf.stack(
        [[pts2[i][j] for i in range(4) for j in range(2)]], axis=0))
    homography = tf.transpose(tf.matrix_solve_ls(a_mat, p_mat, fast=True))
    return homography

In [7]:
def invert_homography(H):
    """
    Computes the inverse transformation for a flattened homography transformation.
    """
    matH = flat2mat(H)
    invH = torch.cat([torch.inverse(matH[i,:,:]) for i in range(matH.shape[0])]).view(matH.shape[0],3,3)
    return mat2flat(invH)

def flat2mat(H):
    """
    Converts a flattened homography with shape '[N, 8]' to its
    corresponding homography matrix with shape '[N, 3, 3]'.
    """
    return torch.reshape(torch.cat((H, torch.ones(H.shape[0],1)), dim=1), [-1,3,3])

def mat2flat(H):
    """
    Converts homography matrix with shape '[N, 3, 3]' to its
    flattened homography with shape '[N, 8]'.
    """
    H = torch.reshape(H, [-1, 9])
    return (H / H[:, 8:9])[:, :8]

# # unit test
# H = torch.ones([10,8])*64
# matH = flat2mat(H)
# flatH = mat2flat(matH)
# print(H)
# print(flatH)

# H = torch.ones([2,8])
# temp = torch.Tensor([1, 0, 3, 2, 1, -1, 4, 2, 4])/4
# H[0,:] = temp[:8]
# H[1,:] = temp[:8]

# print(H)
# print(invert_homography(H))